In [25]:
import pandas as pd
import xml.etree.ElementTree as et
import numpy as np

#### Exploring data sources

- [Web Free Mexico data site ](https://datos.gob.mx/)
- [XML Data sets ](https://datos.gob.mx/busca/dataset/estaciones-de-servicio-gasolineras-y-precios-finales-de-gasolina-y-diesel)


In [26]:
df_places = pd.read_xml('../datasets/places.xml')
df_places.head(20)

,place_id,name,cre_id,location
0,2039,"ESTACION DE SERVICIO CALAFIA, S.A. DE C.V.",PL/658/EXP/ES/2015,NaN
1,2040,"LAS MEJORES ESTACIONES, S.A DE C.V",PL/902/EXP/ES/2015,NaN
2,2041,"DIAZ GAS, S.A. DE C.V.",PL/760/EXP/ES/2015,NaN
3,2042,"COMBU-EXPRESS, S.A. DE C.V.",PL/825/EXP/ES/2015,NaN
4,2043,"PETROMAX, S.A. DE C.V.",PL/585/EXP/ES/2015,NaN
5,2044,"ESTACION RAEL, S. DE R.L. DE C.V.",PL/842/EXP/ES/2015,NaN
6,2045,MULTISERVICIOS LA PILARICA SA DE CV,PL/371/EXP/ES/2015,NaN
7,2046,"PETROMAX, S.A. DE C.V.",PL/439/EXP/ES/2015,NaN
8,2047,"ESTACION RAEL, S. DE R.L. DE C.V.",PL/860/EXP/ES/2015,NaN
9,2048,"ESTACION PIRU, S.A. DE C.V.",PL/897/EXP/ES/2015,NaN


In [27]:
df_places.count()

place_id    13802
name        13802
cre_id      13802
location        0
dtype: int64

In [28]:
df_prices = pd.read_xml('../datasets/prices.xml')
df_prices.head(20)

,place_id,gas_price
0,11703,20.43
1,11703,23.89
2,11702,25.30
3,11701,22.99
4,11700,23.95
5,11699,21.99
6,11699,24.14
7,11698,22.20
8,11698,24.50
9,11697,23.69


In [29]:
df_prices.count()

place_id     22366
gas_price    22366
dtype: int64

### Refactoring data and cleaning data

- Places location (place_id, lat, long )are within xml places data set
- The prices of every gas type of every place (place_id, gas_type) are within xml prices data set.

In order to make the data more readable and handful, It’s necessary to merge datasets and convert it into a pandas data frame.

- [From xml to Pandas](https://datos.gob.mx/busca/dataset/estaciones-de-servicio-gasolineras-y-precios-finales-de-gasolina-y-diesel)


In [30]:
# Refactoring data for places XML file

xtree = et.parse("../datasets/places.xml")
xroot = xtree.getroot()

# Data frame cols and row
df_places_cols = ['place_id','name', 'latitude', 'longitude']
rows = []

# Read xml nodes
for node in xroot:
    place_id = node.attrib.get('place_id')
    name = node.find('name').text if node is not None else None
    location = node.find('location')
    lat = location.find('y').text if location is not None else None
    long = location.find('x').text if location is not None else None

    rows.append({
        'place_id':place_id,
        'name':name,
        'latitude':float(lat),
        'longitude':float(long)
    })

df_places_refactor = pd.DataFrame(rows, columns=df_places_cols)
df_places_refactor.head(20)

,place_id,name,latitude,longitude
0,2039,"ESTACION DE SERVICIO CALAFIA, S.A. DE C.V.",32.47641,-116.92140
1,2040,"LAS MEJORES ESTACIONES, S.A DE C.V",20.30370,-99.74484
2,2041,"DIAZ GAS, S.A. DE C.V.",31.71947,-106.45140
3,2042,"COMBU-EXPRESS, S.A. DE C.V.",20.71413,-103.30420
4,2043,"PETROMAX, S.A. DE C.V.",26.03787,-98.29977
5,2044,"ESTACION RAEL, S. DE R.L. DE C.V.",32.51342,-117.07150
6,2045,MULTISERVICIOS LA PILARICA SA DE CV,25.75580,-108.96850
7,2046,"PETROMAX, S.A. DE C.V.",25.77202,-100.29180
8,2047,"ESTACION RAEL, S. DE R.L. DE C.V.",31.86546,-116.60790
9,2048,"ESTACION PIRU, S.A. DE C.V.",32.31202,-117.04570


In [31]:
df_places_refactor.count()

place_id     13802
name         13802
latitude     13802
longitude    13802
dtype: int64

In [32]:
# Refactoring data for prices XML file

xtree = et.parse("../datasets/prices.xml")
xroot = xtree.getroot()

# Data frame cols and row
df_cols = ['place_id','regular', 'premium', 'diesel']
rows = []

# Read xml nodes
place_ids = dict() # to mange duplicate places
for node in xroot:
    place_id = node.attrib.get('place_id')
    children_iterator = node.find('gas_price').iter()
    gas_types = place_ids[place_id] if place_id in place_ids else {'regular': np.nan, 'premium':np.nan, 'diesel':np.nan}
    for item in children_iterator:
        gas_type = item.attrib.get('type')
        gas_types[gas_type] = item.text

    place_ids[place_id] = gas_types

for place_id,gas_types in place_ids.items():
    rows.append({
        'place_id':place_id,
        'regular':gas_types['regular'],
        'premium':gas_types['premium'],
        'diesel':gas_types['diesel']
    })

df_prices_refactor = pd.DataFrame(rows, columns=df_cols)
df_prices_refactor.head(20)

,place_id,regular,premium,diesel
0,11703,20.43,23.89,NaN
1,11702,24.2,NaN,NaN
2,11701,17.89,NaN,NaN
3,11700,21.75,NaN,NaN
4,11699,21.99,24.14,NaN
5,11698,22.2,NaN,24.5
6,11697,22.09,NaN,23.69
7,11696,21.39,23.49,23.49
8,11695,22.09,NaN,NaN
9,11694,22.09,NaN,NaN


In [33]:
df_prices_refactor.count()

place_id    13303
regular     13243
premium      5102
diesel       3857
dtype: int64

In [34]:
# merging data frames by place_id

df_refactored = df_places_refactor.merge(df_prices_refactor,how='inner')
df_refactored.head(20)

,place_id,name,latitude,longitude,regular,premium,diesel
0,2039,"ESTACION DE SERVICIO CALAFIA, S.A. DE C.V.",32.47641,-116.92140,20.99,24.99,23.39
1,2040,"LAS MEJORES ESTACIONES, S.A DE C.V",20.30370,-99.74484,22.81,24.03,NaN
2,2041,"DIAZ GAS, S.A. DE C.V.",31.71947,-106.45140,17.65,20.07,NaN
3,2042,"COMBU-EXPRESS, S.A. DE C.V.",20.71413,-103.30420,22.14,NaN,NaN
4,2043,"PETROMAX, S.A. DE C.V.",26.03787,-98.29977,18.19,NaN,NaN
5,2044,"ESTACION RAEL, S. DE R.L. DE C.V.",32.51342,-117.07150,21.59,24.99,NaN
6,2045,MULTISERVICIOS LA PILARICA SA DE CV,25.75580,-108.96850,24.05,25.11,NaN
7,2046,"PETROMAX, S.A. DE C.V.",25.77202,-100.29180,22.59,NaN,NaN
8,2047,"ESTACION RAEL, S. DE R.L. DE C.V.",31.86546,-116.60790,21.39,24.69,23.19
9,2048,"ESTACION PIRU, S.A. DE C.V.",32.31202,-117.04570,21.69,24.49,NaN


In [35]:
df_refactored.count()

place_id     13303
name         13303
latitude     13303
longitude    13303
regular      13243
premium       5102
diesel        3857
dtype: int64

In [36]:
# Save into csv
df_refactored.to_csv('refactored_dataset.csv', index=False)